In [1]:
import pandas as pd
import json

tests = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newTestsByPublishDate&format=csv'

cases = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newCasesByPublishDate&format=csv'

tests_scot = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=nation&areaCode=S92000003&metric=newTestsByPublishDate&format=csv'

cases_scot = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=nation&areaCode=S92000003&metric=newCasesByPublishDate&format=csv'


In [2]:
tests_df = pd.read_csv(tests)
cases_df = pd.read_csv(cases)

tests_df = tests_df[['date','newTestsByPublishDate']]
cases_df = cases_df[['date','newCasesByPublishDate']]

combined_df = pd.merge(cases_df,tests_df, on='date', how='left')
combined_df['date'] = pd.to_datetime(combined_df['date'])

combined_df['weekNo'] = combined_df['date'].dt.strftime("%W")
combined_df['year'] = combined_df['date'].dt.year
combined_df['weekNo'] = combined_df['weekNo'].astype(str)
combined_df['year'] = combined_df['year'].astype(str)
combined_df['yyyy-w'] = combined_df['year']+"-"+combined_df['weekNo']

pivot = combined_df.pivot_table(values=['newCasesByPublishDate','newTestsByPublishDate'],index='date',aggfunc=sum).reset_index()
pivot['percentPositive'] = pivot['newCasesByPublishDate']/pivot['newTestsByPublishDate']*100
clean_pivot = pivot[['date','percentPositive','newTestsByPublishDate']]


In [3]:
import plotly.express as px

fig = px.line(combined_df, x="date", y="newCasesByPublishDate", title='Daily Cases UK')
fig.show()

In [4]:
graph_uk = clean_pivot

In [5]:
graph_uk['rollingPositivityRate'] = graph_uk['percentPositive'].rolling(window=7).mean()

In [6]:
graph_uk['year'] = graph_uk['date'].dt.year
graph_uk['dayOfYear'] = graph_uk['date'].dt.day_of_year

fig = px.line(graph_uk, x="dayOfYear", y="rollingPositivityRate", color='year', title='Case Positivity Rate UK')
fig.show()

In [7]:
tests_scot_df = pd.read_csv(tests_scot)
cases_scot_df = pd.read_csv(cases_scot)

tests_scot_df = tests_scot_df[['date','newTestsByPublishDate']]
cases_scot_df = cases_scot_df[['date','newCasesByPublishDate']]

combined_scot_df = pd.merge(cases_scot_df,tests_scot_df, on='date', how='left')
combined_scot_df['percentPositive'] = combined_scot_df['newCasesByPublishDate']/combined_scot_df['newTestsByPublishDate']

combined_scot_df['date'] = pd.to_datetime(combined_scot_df['date'])
combined_scot_df['weekNo'] = combined_scot_df['date'].dt.strftime("%W")
combined_scot_df['year'] = combined_scot_df['date'].dt.year
combined_scot_df['weekNo'] = combined_scot_df['weekNo'].astype(str)
combined_scot_df['year'] = combined_scot_df['year'].astype(str)
combined_scot_df['yyyy-w'] = combined_scot_df['year']+"-"+combined_scot_df['weekNo']

pivot_scot = combined_scot_df.pivot_table(values=['newCasesByPublishDate','newTestsByPublishDate'],index='date',aggfunc=sum).reset_index()
pivot_scot['percentPositive'] = pivot_scot['newCasesByPublishDate']/pivot_scot['newTestsByPublishDate']*100

clean_pivot_scot = pivot_scot[['date','percentPositive']]

clean_pivot_scot['dayOfYear'] = clean_pivot_scot['date'].dt.day_of_year
clean_pivot_scot['year'] = clean_pivot_scot['date'].dt.year

fig = px.line(clean_pivot_scot, x="dayOfYear", y="percentPositive", color = 'year', title='Daily Cases Scotland')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
graph_scot = clean_pivot_scot

graph_scot['year'] = graph_scot['date'].dt.year
graph_scot['dayOfYear'] = graph_scot['date'].dt.day_of_year

graph_scot['rollingPositivityRate'] = graph_scot['percentPositive'].rolling(window=7).mean()

fig = px.line(graph_scot, x="dayOfYear", y="rollingPositivityRate", color = 'year', title='Case Positivity Rate Scotland')
fig.show()
